# Project: Geo-pulse - Prediction of number of interests

---

## 1. Importing Libraries

In [1]:
# General Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sci-kit learn Library
# machine learning models
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# preprocess
from sklearn.model_selection import train_test_split

# metrics
from sklearn.metrics import mean_squared_error

sns.set_palette('colorblind')

In [2]:
df = pd.read_csv('../data/geo_cleaned.csv', index_col=[0])

In [3]:
df.head()

area_code  hour  segment  density  prepaid  postpaid  gender_male  \
0  BAAACBIH    18  visitor       28        9        19            5   
1  BAAACBIH    17  visitor       21       10        11            6   
2  BAAHEBJC     1  visitor       37       17        20           10   
3  BAEFBBAC    16   worker       43       21        22           11   
4  BAJKABJB    12  visitor       21       12         9            8   

   gender_female  gender_unknown  age_1_19  ...  online_shopping_user  \
0             11              12         6  ...                    21   
1              7               8         3  ...                    11   
2              8              19         8  ...                    20   
3              9              23         5  ...                    24   
4              6               7         3  ...                    12   

   inte_travel_and_information  inte_investment  inte_realestate  \
0                            4                3                2   
1                            1                2                0   
2                            1                2                0   
3                            3                3                4   
4                            0                0                0   

   inte_insurance  inte_automobile  inte_fitness_and_wellness   par_day  \
0               0                3                          3  20201104   
1               0                0                          0  20201104   
2               0                4                          3  20201104   
3               3                5                          4  20201104   
4               0                1                          1  20201104   

   area_code_length  time_of_day  
0                 8    Afterwork  
1                 8         Work  
2                 8        Sleep  
3                 8         Work  
4                 8         Work  

[5 rows x 39 columns]

---

## 2. Data Preparation

In [4]:
# drop unused columns
df.drop(columns=['area_code','par_day','area_code_length','hour'], 
        inplace = True)

In [5]:
# defining column groups
GENDER = df.columns[df.columns.str.contains('gender')]
AGE = df.columns[df.columns.str.contains('age')]
PAID = df.columns[df.columns.str.contains('paid')]
PLAN = df.columns[df.columns.str.contains('pay')]
USER = df.columns[df.columns.str.contains('user')]
INTE = df.columns[df.columns.str.contains('inte')]

col_cat = [GENDER,AGE,PAID,PLAN,USER,INTE]
col_cat_name = ['Gender','Age','Post/Pre Paid','Package Plan',
                'Usage','Interests']

---

## 3. Predict number of interests

In [6]:
def get_metrics(model,X_train=1,X_test=1,y_train=1,y_test=1):
    
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_r2 = model.score(X_train, y_train).round(4)
    test_r2 = model.score(X_test, y_test).round(4)
    train_rmse = mean_squared_error(y_train,y_train_pred).round(2)
    test_rmse = mean_squared_error(y_test,y_test_pred).round(2)
    
    return train_r2,test_r2,train_rmse,test_rmse

In [7]:
def predict_category(model_input,category,X_train=1, X_test=1,
                     y_train=1, y_test=1):
    
    results_dict = {}
    
    for col in category:
        
        model = model_input
        
        X = df.drop(columns=category)
        y = df[col]
        X = pd.get_dummies(X,columns=['segment','time_of_day'])

        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size = 0.2,
                                                        random_state=42)
        metrics = []

        train_r2,test_r2,train_rmse,test_rmse = \
        get_metrics(DummyRegressor(),X_train, X_test, y_train, y_test)
        metrics.extend([train_r2,test_r2,train_rmse,test_rmse])


        train_r2,test_r2,train_rmse,test_rmse = \
        get_metrics(model,X_train, X_test, y_train, y_test)
        metrics.extend([train_r2,test_r2,train_rmse,test_rmse])
    
        results_dict[col] = metrics
    
    return pd.DataFrame(results_dict,
                        index=['base_train_r2','base_test_r2','base_train_rmse',
                               'base_test_rmse','model_train_r2',
                               'model_test_r2','model_train_rmse', 
                               'model_test_rmse'])

### 3.1 Linear Regression

#### 3.1.1 Predict number of interests in certain grid

In [8]:
predict_category(LinearRegression(),INTE)

inte_coffee_lover  inte_travel_and_information  \
base_train_r2                0.0000                       0.0000   
base_test_r2                -0.0097                      -0.0006   
base_train_rmse            124.1800                      25.3600   
base_test_rmse             155.1600                      27.6500   
model_train_r2               0.9247                       0.8519   
model_test_r2                0.9247                       0.7202   
model_train_rmse             9.3500                       3.7500   
model_test_rmse             11.5800                       7.7300   

                  inte_investment  inte_realestate  inte_insurance  \
base_train_r2              0.0000           0.0000          0.0000   
base_test_r2              -0.0047          -0.0005         -0.0004   
base_train_rmse           28.5600          10.1400          9.2500   
base_test_rmse            28.7100           8.2900          7.6300   
model_train_r2             0.8478           0.7480          0.7449   
model_test_r2              0.8136           0.5425          0.6971   
model_train_rmse           4.3500           2.5600          2.3600   
model_test_rmse            5.3300           3.7900          2.3100   

                  inte_automobile  inte_fitness_and_wellness  
base_train_r2              0.0000                     0.0000  
base_test_r2              -0.0014                    -0.0024  
base_train_rmse           29.0900                    31.5000  
base_test_rmse            30.7800                    48.2500  
model_train_r2             0.8403                     0.8058  
model_test_r2              0.8332                     0.9117  
model_train_rmse           4.6500                     6.1200  
model_test_rmse            5.1300                     4.2500

### 3.1.2 Predict number of app users in certain grid

In [9]:
predict_category(LinearRegression(),USER)

music_streamer_user  video_streaming_app_user  \
base_train_r2                  0.0000                    0.0000   
base_test_r2                  -0.0019                   -0.0017   
base_train_rmse               59.9600                  325.2000   
base_test_rmse               183.3100                  537.3700   
model_train_r2                 0.9024                    0.9644   
model_test_r2                  0.7261                    0.9480   
model_train_rmse               5.8500                   11.5700   
model_test_rmse               50.1200                   27.8900   

                  merchant_app_user  bank_app_user  fastfood_app_user  \
base_train_r2                0.0000         0.0000             0.0000   
base_test_r2                -0.0024        -0.0024            -0.0000   
base_train_rmse              3.3400       503.5700             4.6100   
base_test_rmse               4.2800       549.3100             4.6700   
model_train_r2               0.5732         0.9701             0.6445   
model_test_r2                0.5863         0.9345             0.6306   
model_train_rmse             1.4300        15.0800             1.6400   
model_test_rmse              1.7700        35.8800             1.7300   

                  food_delivery_app_user  grocery_delivery_app_user  \
base_train_r2                     0.0000                     0.0000   
base_test_r2                     -0.0020                    -0.0008   
base_train_rmse                 226.1800                    87.5800   
base_test_rmse                  270.8200                    80.9500   
model_train_r2                    0.9488                     0.9102   
model_test_r2                     0.9483                     0.8494   
model_train_rmse                 11.5800                     7.8600   
model_test_rmse                  13.9600                    12.1800   

                  online_shopping_user  
base_train_r2                   0.0000  
base_test_r2                   -0.0052  
base_train_rmse               741.4800  
base_test_rmse               1088.8800  
model_train_r2                  0.9825  
model_test_r2                   0.9855  
model_train_rmse               12.9400  
model_test_rmse                15.6700

### 3.2 XGBoost

#### 3.2.1 Predict number of interests in certain grid

In [10]:
predict_category(XGBRegressor(),INTE)

inte_coffee_lover  inte_travel_and_information  \
base_train_r2                0.0000                       0.0000   
base_test_r2                -0.0097                      -0.0006   
base_train_rmse            124.1800                      25.3600   
base_test_rmse             155.1600                      27.6500   
model_train_r2               1.0000                       0.9999   
model_test_r2                0.8689                       0.8042   
model_train_rmse             0.0000                       0.0000   
model_test_rmse             20.1500                       5.4100   

                  inte_investment  inte_realestate  inte_insurance  \
base_train_r2              0.0000           0.0000          0.0000   
base_test_r2              -0.0047          -0.0005         -0.0004   
base_train_rmse           28.5600          10.1400          9.2500   
base_test_rmse            28.7100           8.2900          7.6300   
model_train_r2             0.9999           0.9999          0.9999   
model_test_r2              0.7729           0.6142          0.5834   
model_train_rmse           0.0000           0.0000          0.0000   
model_test_rmse            6.4900           3.2000          3.1800   

                  inte_automobile  inte_fitness_and_wellness  
base_train_r2              0.0000                     0.0000  
base_test_r2              -0.0014                    -0.0024  
base_train_rmse           29.0900                    31.5000  
base_test_rmse            30.7800                    48.2500  
model_train_r2             0.9999                     0.9999  
model_test_r2              0.7942                     0.6672  
model_train_rmse           0.0000                     0.0000  
model_test_rmse            6.3300                    16.0200

#### 3.2.2 Predict number of app users in certain grid

In [11]:
predict_category(XGBRegressor(),USER)

music_streamer_user  video_streaming_app_user  \
base_train_r2                  0.0000                    0.0000   
base_test_r2                  -0.0019                   -0.0017   
base_train_rmse               59.9600                  325.2000   
base_test_rmse               183.3100                  537.3700   
model_train_r2                 0.9999                    1.0000   
model_test_r2                  0.4682                    0.7764   
model_train_rmse               0.0000                    0.0100   
model_test_rmse               97.3100                  119.9600   

                  merchant_app_user  bank_app_user  fastfood_app_user  \
base_train_r2                0.0000         0.0000             0.0000   
base_test_r2                -0.0024        -0.0024            -0.0000   
base_train_rmse              3.3400       503.5700             4.6100   
base_test_rmse               4.2800       549.3100             4.6700   
model_train_r2               0.9998         1.0000             0.9998   
model_test_r2                0.5124         0.9431             0.4926   
model_train_rmse             0.0000         0.0100             0.0000   
model_test_rmse              2.0800        31.2100             2.3700   

                  food_delivery_app_user  grocery_delivery_app_user  \
base_train_r2                     0.0000                     0.0000   
base_test_r2                     -0.0020                    -0.0008   
base_train_rmse                 226.1800                    87.5800   
base_test_rmse                  270.8200                    80.9500   
model_train_r2                    1.0000                     1.0000   
model_test_r2                     0.8897                     0.8605   
model_train_rmse                  0.0100                     0.0000   
model_test_rmse                  29.8000                    11.2900   

                  online_shopping_user  
base_train_r2                   0.0000  
base_test_r2                   -0.0052  
base_train_rmse               741.4800  
base_test_rmse               1088.8800  
model_train_r2                  1.0000  
model_test_r2                   0.8715  
model_train_rmse                0.0000  
model_test_rmse               139.1800